In [2]:
import json
import requests
import pickle

In [3]:
import pandas as pd
import numpy as np
import os
import random

# Using SciPy for distance.



In [4]:
import math 

import scipy.spatial.distance

def ruin_distance(ruin1, ruin2):
    return scipy.spatial.distance.euclidean(ruin1['systemCoordinates'], ruin2['systemCoordinates'])



# Canonn Technology API Documentation is here

https://api.canonn.technology/api/docs/#/

This section fetches all of the data and provides helper functions

In [5]:
api_url_base = 'https://api.canonn.technology/api/v1/'

#headers = {'Content-Type': 'application/json'}


In [6]:
api_url_base = 'https://api.canonn.technology/api/v1/'

def get_from_site(url_ext):
    """Grabs data from the API / url_ext
    Returns None if 200. not much other error handling 
    """
    api_url = '{0}{1}'.format(api_url_base,url_ext)

    response = requests.get(api_url)

    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

In [7]:
def get_obelisks():
    return get_from_site('maps/scandata')

def get_ruins_sites():
    return get_from_site('maps/systemoverview')

def get_categories():
    return get_from_site('codex/categories')

def get_codex_data():
    return get_from_site('codex/data')

def get_ruin_types():
    return get_from_site('ruins/types')

def get_all_ruins():
    return get_from_site('ruinsites')

In [8]:
def get_ruin_info(id):
    return get_from_site('maps/ruininfo/%d' % id) 

def get_active_obelisks(id):
    return get_from_site('ruinsites/%d/activeobelisks' % id)

In [9]:
ruin_types = get_ruin_types()
ruins = get_ruins_sites()
obelisks = get_obelisks()
codex_data = get_codex_data()

# all_codex_data is the set of all codex data that we need to find 
# in a set so we can subtract from it along the way to find what's left
all_codex_data = set([x['id'] for x in codex_data])

all_ruins = get_all_ruins()

In [10]:
# Need to filter out all ruin id's that are >10000 - or just drop the last 3 that are special
# Could also filter out edsmBodyLink = None 
# Risky -- says that no ruin will be exactly at longitude 0.0 .. probably good enough 

all_ruins_trimmed = [x for x in all_ruins if (x['longitude'] != 0.0)]


In [19]:
def get_obelisk_group_codex_data(id):
    """Relies upon all_obelisk_info being populated
    returns the sorted set of codex data that this particular obelisk group handles
    """
    obelisk_group = all_obelisk_df[all_obelisk_df['ruinId'] == id]['obeliskInfo']
    for i in obelisk_group.iteritems():
        if i[1] != None:
            return sorted(set([x['codexdataId'] for x in i[1] if (x['isBroken'] == False and x['codexdataId'] != None)])) 
        else:
            return None
        
def count_active_obelisks(id):
    items = get_obelisk_group_codex_data(id)
    if items == None:
        return 0
    else:
        return len(items)


In [22]:
def load_all_ruins_info(force=False):
    """Loads all of the info about the ruins and obelisks, 
    Either from disk (if available) or fetching from the web
    when it fetchs from the web it is slow
    the force argument requires fetching from online."""
    if force or not os.path.exists("ruins.p"):
        # Long running command - probably should rate limit the get API.
        all_ruin_info = [get_ruin_info(ruin['id']) for ruin in all_ruins_trimmed]
        pickle.dump( all_ruin_info, open( "ruins.p", "wb" ) )
    else:
        all_ruin_info = pickle.load( open( "ruins.p", "rb" ) )
    return all_ruin_info

def load_all_obelisks_info(force=False):
    if force or not os.path.exists("obelisks.p"):
        # Long running command - probably should rate limit the get API.
        all_obelisk_info = [get_active_obelisks(ruin['id']) for ruin in all_ruins_trimmed]    
        pickle.dump( all_obelisk_info, open( "obelisks.p", "wb"))
    else:
        all_obelisk_info = pickle.load( open ( "obelisks.p", "rb"))
    return all_obelisk_info
        
def load_obelisk_dataframe(force=False):
    if force or not os.path.exists("all_obelisk_df.p"):
        all_obelisk_df = pd.DataFrame(load_all_ruins_info())
        # Save the result in a new column
        all_obelisk_df['obeliskInfo'] = all_obelisk_df['ruinId'].apply(get_active_obelisks)
        pickle.dump( all_obelisk_df, open ("all_obelisk_df.p", "wb"))
    else: 
        all_obelisk_df = pickle.load( open ("all_obelisk_df.p", "rb"))
    return all_obelisk_df


all_obelisk_df = load_obelisk_dataframe()
all_obelisk_df['obeliskCount'] = all_obelisk_df['ruinId'].apply(count_active_obelisks)
all_obelisk_df['obeliskList'] = all_obelisk_df['ruinId'].apply(get_obelisk_group_codex_data)


In [23]:
print(get_obelisk_group_codex_data(156))
for i in range(len(all_ruins_trimmed)):
    print(get_obelisk_group_codex_data(i))
    
    
import random
group_of_items = range(len(all_ruin_info))              # a sequence or set will work here.
num_to_select = 10                           # set the number to select here.
list_of_random_items = random.sample(group_of_items, num_to_select)
print(list_of_random_items)

setup = all_codex_data
for ruin in list_of_random_items:
    ob = get_obelisk_group_codex_data(ruin)
    setup = setup - set(ob)
    print(ob)
    
print(setup)

[5, 6, 21, 22, 23, 25, 42, 43, 44, 45, 63, 64, 67, 86]
None
[47, 48, 49, 50, 52, 70, 72, 89, 90]
[8, 27, 28, 54, 55, 88, 90, 91]
[5, 6, 20, 21, 24, 25, 40, 42, 43, 44]
[29, 32, 52, 54, 68, 90, 93]
[1, 2, 4, 22, 23, 24, 25, 42, 44, 64, 67, 84, 85]
[26, 48, 51, 71, 72, 88, 90, 91]
[2, 6, 20, 21, 23, 40, 41, 44, 63, 64, 65, 66, 84, 85, 86, 87]
[26, 27, 31, 47, 54, 88, 90, 91]
[13, 14, 15, 17, 18, 33, 34, 35, 36, 38, 56, 57, 59, 60, 78, 95, 98, 99, 100]
[8, 27, 49, 50, 73, 74, 88, 90, 94]
[23, 24, 25, 40, 42, 43, 44, 45, 62, 63, 64, 65, 82, 83, 84, 85, 86]
[2, 3, 6, 20, 22, 23, 24, 25, 40, 42, 45, 62, 63, 66, 82, 83, 84, 87]
[3, 4, 6, 21, 22, 25, 45, 46, 63, 64, 65, 84, 86]
[14, 16, 17, 18, 19, 35, 36, 37, 38, 56, 57, 59, 60, 95, 96, 97, 99, 100, 101]
[13, 14, 19, 34, 37, 38, 39, 56, 59, 75, 76, 78, 79, 81, 97, 98, 99]
[15, 17, 35, 36, 37, 56, 59, 95, 96, 97, 99, 100, 101]
[28, 31, 48, 49, 53, 89, 93, 94]
[29, 48, 49, 53, 72, 89, 92, 94]
[29, 47, 51, 52, 53, 55, 73, 88, 90]
[15, 16, 38, 39

In [31]:
print (all_obelisk_df['obeliskCount'].mean())

all_obelisk_df.loc[all_obelisk_df['obeliskCount'] == all_obelisk_df['obeliskCount'].max()]

# These are all of the locations that have >2* STD over mean active obelisks
all_obelisk_df.loc[all_obelisk_df['obeliskCount'] >= all_obelisk_df['obeliskCount'].mean() + 2*all_obelisk_df['obeliskCount'].std()]#['ruinId']


12.31858407079646


,bodyDistance,bodyId,bodyName,coordinates,edsmBodyLink,edsmSystemLink,obelisks,ruinId,ruinTypeName,systemCoordinates,systemId,systemName,obeliskInfo,obeliskCount,obeliskList
26,4751.0,15,AB 9 B,"[21.9117, 154.176]",https://www.edsm.net/en/system/bodies/id/91881...,https://www.edsm.net/en/system/id/9188152/name...,"{'a': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0,...",27,Gamma,"[1120.34, -87.2188, -216.875]",12,COL 173 SECTOR LY-Q D5-13,"[{'id': 624, 'obeliskgroupId': 39, 'number': 1...",21,"[13, 14, 17, 33, 34, 35, 37, 38, 39, 56, 57, 5..."
43,69360.0,24,D 2,"[28.8065, -51.8289]",https://www.edsm.net/en/system/bodies/id/88510...,https://www.edsm.net/en/system/id/8851093/name...,"{'a': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0,...",44,Alpha,"[811.406, -60.4375, -144.719]",18,SYNUEFE ZR-I B43-10,"[{'id': 6, 'obeliskgroupId': 1, 'number': 6, '...",22,"[1, 2, 3, 4, 6, 21, 22, 23, 24, 25, 40, 41, 43..."
148,691.0,97,B 2,"[27.5997, -105.8709]",https://www.edsm.net/en/system/bodies/id/12951...,https://www.edsm.net/en/system/id/12951546/nam...,"{'a': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0,...",149,Gamma,"[1044.69, -158.438, -62.5625]",82,VELA DARK REGION WT-R B4-3,"[{'id': 617, 'obeliskgroupId': 39, 'number': 7...",20,"[13, 15, 17, 18, 34, 35, 36, 37, 38, 56, 57, 7..."
188,2589.0,144,C 3,"[8.0827, -48.3564]",https://www.edsm.net/en/system/bodies/id/14354...,https://www.edsm.net/en/system/id/14354875/nam...,"{'b': {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0,...",189,Gamma,"[1305.19, -169.125, -52.125]",120,COL 173 SECTOR VN-A B30-1,"[{'id': 645, 'obeliskgroupId': 40, 'number': 8...",20,"[13, 14, 16, 34, 35, 36, 38, 56, 57, 75, 76, 7..."
191,NaN,147,C 2,"[-52.36, 167.95]",https://www.edsm.net/en/system/bodies/id/15121...,https://www.edsm.net/en/system/id/15121599/nam...,"{'a': {'1': 0, '2': 0, '3': 1, '4': 0, '5': 0,...",192,Gamma,"[-5476.69, -518.812, 10526.7]",123,PRUA PHOE US-B D86,"[{'id': 613, 'obeliskgroupId': 39, 'number': 3...",21,"[14, 15, 18, 19, 33, 36, 38, 39, 56, 57, 58, 6..."


In [51]:
def check_degree_overlap(ruinId1, ruinId2):
    ob1 = all_obelisk_df.loc[all_obelisk_df['ruinId'] == ruinId1]
    ob2 = all_obelisk_df.loc[all_obelisk_df['ruinId'] == ruinId2]
    print(all_obelisk_df[ob1]['obeliskList'])
    print(list(ob2['obeliskList']))
    print(ob1['obeliskList'] & ob2['obeliskList'])

check_degree_overlap(27,44)

ValueError: Must pass DataFrame with boolean values only

# Start processing Data

In [25]:
print(get_ruin_info(158))

{'edsmBodyLink': 'https://www.edsm.net/en/system/bodies/id/187704/name/2MASS+J10444160-5947046/details/idB/5170678/nameB/2MASS+J10444160-5947046+1+B', 'ruinId': 158, 'bodyName': '1 B', 'systemCoordinates': [8614.19, -116.688, 2733.03], 'bodyDistance': 1541, 'edsmSystemLink': 'https://www.edsm.net/en/system/id/187704/name/2MASS+J10444160-5947046', 'coordinates': [-28.52, -156.7], 'obelisks': {'i': {'9': 0, '11': 0, '15': 0, '17': 0, '4': 0, '7': 1, '19': 0, '14': 0, '12': 0, '3': 0, '2': 0, '13': 0, '5': 0, '18': 0, '20': 0, '10': 0, '1': 0, '16': 0, '6': 0, '8': 0}, 'h': {'7': 0, '3': 0, '2': 0, '5': 1, '4': 0, '1': 0, '6': 0}, 'b': {'9': 1, '11': 0, '15': 0, '17': 0, '4': 0, '7': 0, '19': 0, '14': 0, '21': 0, '12': 1, '3': 0, '2': 0, '13': 0, '5': 0, '18': 0, '20': 0, '10': 0, '1': 0, '16': 0, '6': 0, '8': 0}, 'e': {'9': 0, '7': 1, '3': 0, '2': 0, '5': 0, '4': 0, '10': 0, '1': 0, '6': 0, '8': 0}, 'k': {'17': 0, '15': 0, '22': 0, '19': 0, '16': 0, '21': 0, '10': 0, '3': 0, '11': 0, '13

In [26]:
print(ruins[5]['systemName'])
ruin1 = get_ruin_info(158)
ruin2 = get_ruin_info(156)
print(ruin1['systemCoordinates'])
print(ruin2['systemCoordinates'])

print(ruin_distance(ruin1, ruin2))


COL 173 SECTOR AP-Q B21-2
[8614.19, -116.688, 2733.03]
[8629.19, -98.8125, 2713.06]
30.713749368157746


In [11]:
obelisks_260 = get_active_obelisks(226)


In [85]:
print(obelisks_260)
print(len(obelisks_260))
print([x['codexdataId'] for x in obelisks_260 if x['isBroken'] == False])

[{'codexdataId': 19, 'created': '2017-03-19T20:59:30', 'isVerified': False, 'obeliskgroupId': 40, 'isBroken': False, 'updated': '2017-03-19T20:59:30', 'number': 3, 'id': 640}, {'codexdataId': 18, 'created': '2017-03-19T20:59:30', 'isVerified': True, 'obeliskgroupId': 40, 'isBroken': False, 'updated': '2017-07-11T18:21:04', 'number': 4, 'id': 641}, {'codexdataId': 78, 'created': '2017-03-19T20:59:30', 'isVerified': False, 'obeliskgroupId': 42, 'isBroken': False, 'updated': '2017-07-11T18:21:04', 'number': 3, 'id': 682}, {'codexdataId': 59, 'created': '2017-03-19T20:59:30', 'isVerified': True, 'obeliskgroupId': 43, 'isBroken': False, 'updated': '2017-07-11T18:21:04', 'number': 5, 'id': 700}, {'codexdataId': 100, 'created': '2017-03-19T20:59:30', 'isVerified': True, 'obeliskgroupId': 45, 'isBroken': False, 'updated': '2017-07-11T18:21:04', 'number': 13, 'id': 733}, {'codexdataId': 17, 'created': '2017-03-19T20:59:30', 'isVerified': True, 'obeliskgroupId': 47, 'isBroken': False, 'updated':

In [13]:
print(get_obelisk_group_codex_data(15))

[19, 18, 78, 59, 100, 17, 15, 59, 99, 101, 33, 37, 95, 97, 36, 56]


In [14]:
print(codex_data)
all_codex_data = set([x['id'] for x in codex_data])
print(all_codex_data)

[{'id': 1, 'categoryId': 1, 'artifactId': 3, 'entryNumber': 1, 'text': "Thank you commander, this data has proven vital to my ongoing research. While it seemed logical that the alpha-pattern data corresponded with the core information on the Guardians' biology and ecology, I couldn't be certain until I received the data from your scan. I'm pleased my hypothesis has proved correct, and I've started to unlock some fascinating details about the Guardians. The extracted data relates to individuals rather than their species as a whole, but thanks to the information you have transmitted, I believe I will be able to merge the data to create a general picture.", 'created': '2017-03-19T20:59:29', 'updated': '2017-04-23T16:37:04'}, {'id': 2, 'categoryId': 1, 'artifactId': 6, 'entryNumber': 2, 'text': "The information you've uncovered is very interesting and regards the Guardians' anatomy and physiology, you might be surprised to learn they weren't that different to us. Their morphology was biped

In [15]:
ob15 = get_obelisk_group_codex_data(15)
print (all_codex_data - set(ob15))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 98}


In [38]:
all_locations_info = [x['systemCoordinates'] for x in all_ruin_info]


#"""Create the distance matrix."""
size = len(all_locations_info)
matrix = {}

for from_node in range(size):
  matrix[from_node] = {}
  for to_node in range(size):
     if from_node == to_node:
          matrix[from_node][to_node] = 0
     else:
          matrix[from_node][to_node] = scipy.spatial.distance.euclidean(all_locations_info[from_node], all_locations_info[to_node])
        



[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, -74.75]
[357.344, -49.3438, 

[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[834.219, -51.2188, -154.656]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.7

[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100.75, -246.062]
[1043.88, -100

[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[866.594, -119.125, -109.031]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, -87.2188, -216.875]
[1120.34, 

[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, -124.031, -216.812]
[1150.75, 

...

[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84, -548.031, 10516.9]
[-5541.84,

[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56, -559.094, 10547.7]
[-5525.56,

[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69, -518.812, 10526.7]
[-5476.69,

[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221.344, 167.281]
[1172.03, -221

[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, -175.625, -48.3125]
[1293.56, 

In [41]:
print(all_ruin_info[0])


{'ruinId': 1, 'bodyName': '1 B', 'systemCoordinates': [357.344, -49.3438, -74.75], 'bodyDistance': 803, 'edsmBodyLink': 'https://www.edsm.net/en/system/bodies/id/6379187/name/SYNUEFE+XR-H+D11-102/details/idB/34490/nameB/SYNUEFE+XR-H+D11-102+1+B', 'coordinates': [-31.7347, -128.9212], 'obelisks': {'c': {'22': 0, '15': 1, '23': 0, '2': 0, '19': 0, '16': 0, '21': 0, '10': 0, '3': 0, '9': 1, '13': 0, '5': 0, '20': 0, '12': 0, '1': 0, '17': 0, '4': 0, '7': 0, '14': 0, '11': 0, '18': 0, '25': 0, '6': 0, '24': 0, '8': 0}, 'q': {'7': 0, '3': 0, '2': 0, '5': 0, '4': 1, '1': 0, '6': 0, '8': 0}, 't': {'9': 0, '17': 0, '15': 0, '4': 0, '7': 0, '2': 0, '14': 0, '10': 0, '3': 0, '11': 0, '13': 0, '5': 0, '18': 1, '12': 0, '1': 0, '16': 1, '6': 0, '8': 0}, 'a': {'9': 0, '3': 0, '2': 0, '5': 0, '7': 0, '10': 0, '4': 0, '1': 0, '6': 0, '8': 1}, 'j': {'9': 0, '3': 1, '2': 0, '5': 0, '7': 0, '10': 0, '4': 0, '1': 0, '6': 0, '8': 0}, 'g': {'9': 0, '17': 0, '15': 0, '4': 0, '7': 0, '2': 0, '14': 0, '10': 0

In [88]:
print(df.head)

<bound method NDFrame.head of      bodyDistance  bodyId bodyName            coordinates  \
0           803.0       1      1 B  [-31.7347, -128.9212]   
1          2428.0       2      B 1   [-29.1664, -30.5041]   
2        337757.0       3      C 3        [29.42, -59.54]   
3          1458.0       4      C 1       [19.53, -141.76]   
4          1070.0       5      9 B   [-23.3821, 178.9094]   
5          1014.0       6      8 C     [16.2701, 18.1282]   
6          1014.0       6      8 C       [46.07, -171.33]   
7          6861.0       7   A 14 F     [7.0924, 170.2272]   
8          1186.0       8     CD 1          [59.33, 15.1]   
9          1186.0       8     CD 1        [31.96, -99.97]   
10         1186.0       8     CD 1       [31.88, -100.05]   
11         1443.0       9      9 A  [-15.0842, -102.9397]   
12         2026.0      10      B 3        [-30.54, -24.2]   
13         1443.0       9      9 A    [-37.5256, 84.2501]   
14         1828.0      11      C 2        [-10.07, 114.

In [165]:
import random
group_of_items = range(len(all_ruin_info))              # a sequence or set will work here.
num_to_select = 10                           # set the number to select here.
list_of_random_items = random.sample(group_of_items, num_to_select)
print(list_of_random_items)

setup = all_codex_data
for ruin in list_of_random_items:
    ob = get_obelisk_group_codex_data(ruin)
    setup = setup - set(ob)
    print(ob)
    
print(setup)

[125, 87, 58, 25, 70, 36, 96, 109, 111, 5]
[1, 2, 5, 20, 21, 22, 23, 25, 45, 62, 63, 67, 82, 84, 87]
[13, 15, 19, 33, 35, 37, 39, 56, 57, 58, 75, 76, 78, 79, 80, 99, 101]
[28, 30, 49, 52, 54, 88, 90]
[7, 8, 9, 28, 29, 32, 69, 70, 72, 88, 90, 92]
[7, 12, 30, 32, 48, 54, 55, 88, 90, 91]
[15, 19, 36, 39, 56, 57, 59, 79, 96, 97, 98, 99]
[8, 11, 27, 29, 30, 49, 50, 73, 90, 91]
[4, 5, 20, 24, 40, 44, 46, 61, 63, 65, 82, 84, 86]
[2, 3, 6, 20, 23, 40, 41, 42, 46, 62, 63, 65, 85, 86, 87]
[1, 2, 4, 22, 23, 24, 25, 42, 44, 64, 67, 84, 85]
{10, 14, 16, 17, 18, 26, 31, 34, 38, 43, 47, 51, 53, 60, 66, 68, 71, 74, 77, 81, 83, 89, 93, 94, 95, 100}
